In [2]:
%matplotlib inline
import cirq as cirq
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'cirq'

# quantumMatrixProductStates
This notebook walks through how we represent, optimise, and time evolve uniform matrix product states on a NISQ quantum computer. It depends on xmps, a matrix product state library available at https://github.com/fergusbarratt/xmps, and Cirq, google's quantum programming language

## Intro to uniform Matrix Product States

An uniform matrix product state in right canonical form can be represented by a single repeated tensor

In [ ]:
"""
...--A--A--A--A--A--A--...
  |  |  |  |  |  |
""";

Since we can put the above into left canonical form by a gauge transformation, the expectation value of any local observable can be expressed as follows

In [ ]:
"""
         --A--...--A--
        |  |       |  |
<O>   = |  ----O----  r
        |  |       |  |
         --A--...--A--

""";

Where r is the largest right eigenvector of the transfer matrix

In [ ]:
"""
--A--    ---
  |  |      |
  |  r =    r
  |  |      |
--A--    ---

""";

## Representation

In order to represent a uniform matrix product state of some bond dimension, two objects are sufficient

 - The state tensor A
 - The right eigenmatrix r
 
The main idea of this paper is that these can be represented or determined efficiently on a quantum computer, and the resulting quantum state, representing the state of a infinite, TI quantum system, can be optimized and manipulated in an efficient manner.

#### State Tensor

The A tensor of a matrix product state of bond dimension D can be represented on $\log_2(D)+1$ qubits as follows

In [3]:
"""       
            |0>|0>...|0>|0>
               \    j     /
i--A--j  =   |  |   ...  |
   |         -------------
   σ               U
             -------------
             |   ...  |  | 
            /    i     \/σ\
              \log_2(D)  +1
""";

Where we assume that the state unitary $U$ can be efficiently represented on a quantum computer.
In code this is represented by the
`
StateTensor
`
object.

#### Environment

The environment tensor is embedded in a unitary $V$ as follows

In [4]:
"""
            |0>|0>...|0>|0>
i--r--j  =   |  | ... |  |
             -------------
                   v
             -------------
             |  | ... |  | 
            /      \/     \
           \log_2(D)\log_2(D)
""";

Where we assume that the state unitary $V$ can be efficiently represented on a quantum computer.
In code this is represented by the
`
Environment
`
object.

#### State

All q-local observables of the infinite TI system can be obtained by measurements of corresponding pauli string on the following combination of $U$ and $V$ on the qubits marked $1 \ldots q$

In [5]:
"""
         --A--...--A--
        |  |       |  |
<O>   = |  ----O----  r
        |  |       |  |
         --A--...--A--

      =
        |0>......................|0>
         | | | ... | | ... || ... |
         | | | ... | ---------------
         | | | ... |        V         
         | | | ... | ---------------           
         | | | ... | | ... || ... |
         | | | ... | | ... |
         | | | ... ---------
         | | | ...     U            
         | | | ... ---------            
         | | | ... | ... | | 
         | | | .........   q
         | | | .........
         | | | ... |
         | ---------
         |     U            
         | ---------            
         | | ... | | 
         | | ... | 2
         ---------
             U            
         ---------            
         | ... | | 
               1
            
""";

and require the creation of a circuit on $(q+1)\log_2(D)$ qubits.

In code the `State` class is the combination of a `StateTensor` and `Environment` object, and takes q as an argument $q$.